# Controlling Kubernetes from Python

In [1]:
from kubernetes import client as kclient
from kubernetes import config as kconfig
import yaml
import os

In [2]:
nginxdep = {'apiVersion': 'extensions/v1beta1',
            'kind': 'Deployment',
            'metadata': {'name': 'my-nginx'},
            'spec': {'replicas': 1,
                     'selector': {'matchLabels': {'run': 'my-nginx'}},
                     'template': {'metadata': {'labels': {'run': 'my-nginx'}},
                                  'spec': {'containers': [{'image': 'nginx',
                                                           'name': 'my-nginx',
                                                           'ports': [{'containerPort': 80}]}]}}}}
nginxdep

{'apiVersion': 'extensions/v1beta1',
 'kind': 'Deployment',
 'metadata': {'name': 'my-nginx'},
 'spec': {'replicas': 1,
  'selector': {'matchLabels': {'run': 'my-nginx'}},
  'template': {'metadata': {'labels': {'run': 'my-nginx'}},
   'spec': {'containers': [{'image': 'nginx',
      'name': 'my-nginx',
      'ports': [{'containerPort': 80}]}]}}}}

In [3]:
nginxsvc = {'apiVersion': 'v1',
            'kind': 'Service',
            'metadata': {'labels': {'run': 'my-nginx'}, 'name': 'my-nginx'},
            'spec': {'externalIPs': ['35.200.42.99'],
                     'ports': [{'nodePort': 30000, 'port': 80, 'protocol': 'TCP'}],
                     'selector': {'run': 'my-nginx'},
                     'type': 'LoadBalancer'}}
nginxsvc

{'apiVersion': 'v1',
 'kind': 'Service',
 'metadata': {'labels': {'run': 'my-nginx'}, 'name': 'my-nginx'},
 'spec': {'externalIPs': ['35.200.42.99'],
  'ports': [{'nodePort': 30000, 'port': 80, 'protocol': 'TCP'}],
  'selector': {'run': 'my-nginx'},
  'type': 'LoadBalancer'}}

In [4]:
with open("kube/nginxdep.yml", "w") as f:
    f.write(yaml.dump(nginxdep, default_flow_style=False))

In [5]:
with open("kube/nginxsvc.yml", "w") as f:
    f.write(yaml.dump(nginxsvc, default_flow_style=False))

In [6]:
 class KubeDeployer(object):
    def __init__(self):
        kconfig.load_kube_config()
        self.__k8s_beta = kclient.ExtensionsV1beta1Api()
        self.__k8s_core = kclient.CoreV1Api()
    def createDeployment(self, filename, filedir=None):
        if filedir is not None:
            filename = os.path.join(filedir, filename)
        with open(filename) as f:
            dep = yaml.load(f)
            resp = self.__k8s_beta.create_namespaced_deployment(
                body=dep, namespace="default")
            print("DEPLOYMENT {0} created. status={1}".format(filename, resp.metadata))
        return resp
    def getDeploymentList(self):
        resp = self.__k8s_beta.list_deployment_for_all_namespaces()
        return resp
    def deleteDeployment(self, name, namespace="default", **kwargs):
        body = kubernetes.client.V1DeleteOptions(**kwargs)
        resp = self.__k8s_beta.delete_namespaced_deployment(name, namespace, body, **kwargs)
        return resp
    def deleteAllDeployment(self, namespace="default", **kwargs):
        respd = self.__k8s_beta.delete_collection_namespaced_deployment(namespace)
        respr = self.__k8s_beta.delete_collection_namespaced_replica_set(namespace)
        respp = self.__k8s_core.delete_collection_namespaced_pod(namespace)
        return respd,respr,respp

    def createService(self, filename, filedir=None):
        if filedir is not None:
            filename = os.path.join(filedir, filename)
        with open(filename) as f:
            svc = yaml.load(f)            
            resp = self.__k8s_core.create_namespaced_service(
                body=svc, namespace="default")
            print("SERVICE {0} created. status={1}".format(filename, resp.metadata))
        return resp
    def getServiceList(self):
        resp = self.__k8s_core.list_service_for_all_namespaces()
        return resp
    def deleteService(self, name, namespace="default", **kwargs):
        resp = self.__k8s_core.delete_namespaced_service(name, namespace, **kwargs)
        return resp

In [7]:
kd = KubeDeployer()

In [8]:
kdeploy = kd.createDeployment("nginxdep.yml", "/opt/workdir/kube/")

DEPLOYMENT /opt/workdir/kube/nginxdep.yml created. status={'annotations': None,
 'cluster_name': None,
 'creation_timestamp': datetime.datetime(2018, 2, 4, 6, 51, 36, tzinfo=tzlocal()),
 'deletion_grace_period_seconds': None,
 'deletion_timestamp': None,
 'finalizers': None,
 'generate_name': None,
 'generation': 1,
 'initializers': None,
 'labels': {'run': 'my-nginx'},
 'name': 'my-nginx',
 'namespace': 'default',
 'owner_references': None,
 'resource_version': '15108',
 'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/my-nginx',
 'uid': 'd814551e-0977-11e8-b962-42010a920004'}


In [9]:
ksvc = kd.createService("nginxsvc.yml", "/opt/workdir/kube/")

SERVICE /opt/workdir/kube/nginxsvc.yml created. status={'annotations': None,
 'cluster_name': None,
 'creation_timestamp': datetime.datetime(2018, 2, 4, 6, 51, 37, tzinfo=tzlocal()),
 'deletion_grace_period_seconds': None,
 'deletion_timestamp': None,
 'finalizers': None,
 'generate_name': None,
 'generation': None,
 'initializers': None,
 'labels': {'run': 'my-nginx'},
 'name': 'my-nginx',
 'namespace': 'default',
 'owner_references': None,
 'resource_version': '15125',
 'self_link': '/api/v1/namespaces/default/services/my-nginx',
 'uid': 'd8a45dad-0977-11e8-b962-42010a920004'}
